# STR_001_ORB - Strategy Engine

## Opening Range Breakout Strategy Execution

This notebook demonstrates the strategy execution framework using STR_001_ORB.

In [ ]:
# Import required modules
import sys
from pathlib import Path

# Add project to path
project_root = Path().absolute()
sys.path.insert(0, str(project_root))

from itos.core.data_engine import DataEngine
from itos.core.strategy_engine import StrategyEngine
from itos.core.analytics_engine import AnalyticsEngine
from itos.strategies.str001_orb import STR001_ORB

import pandas as pd
import matplotlib.pyplot as plt

print("✅ Modules imported successfully")

## Strategy Configuration

In [ ]:
# Initialize strategy
strategy = STR001_ORB()
config = strategy.get_config()

print("📋 Strategy Configuration:")
print(f"   Name: {config.name}")
print(f"   Side: {config.side}")
print(f"   Entry Window: {config.entry_time_start} - {config.entry_time_end}")
print(f"   Exit Time: {config.exit_time}")
print(f"   Parameters: {config.parameters}")
print(f"   Description: {config.description}")

## Load Market Data

In [ ]:
# Initialize data engine
data_engine = DataEngine()

# Get available symbols
symbols = data_engine.get_processed_symbols()
print(f"📊 Available symbols: {symbols}")

# Load data for one symbol to test
if symbols:
    test_symbol = symbols[0]
    market_data = data_engine.load_symbol_data(test_symbol)
    
    if market_data:
        print(f"\n📈 Loaded data for {test_symbol}:")
        print(f"   Shape: {market_data.data.shape}")
        print(f"   Date range: {market_data.data.index.min()} to {market_data.data.index.max()}")
        print(f"   Columns: {list(market_data.data.columns)}")
        
        # Display sample data
        display(market_data.data.head())
    else:
        print(f"❌ No data found for {test_symbol}")
else:
    print("❌ No symbols found. Please generate sample data first.")

## Test Strategy on Single Symbol

In [ ]:
# Test strategy on single symbol
if symbols and market_data:
    print(f"🧪 Testing {config.name} on {test_symbol}...")
    
    # Generate signals
    trades = strategy.generate_signals(market_data)
    
    print(f"\n📊 Generated {len(trades)} trades")
    
    if trades:
        # Convert trades to DataFrame for analysis
        trades_data = []
        for trade in trades:
            trades_data.append({
                'symbol': trade.symbol,
                'strategy': trade.strategy,
                'entry_time': trade.entry_time,
                'exit_time': trade.exit_time,
                'entry_price': trade.entry_price,
                'exit_price': trade.exit_price,
                'quantity': trade.quantity,
                'side': trade.side,
                'pnl': trade.pnl,
                'exit_reason': trade.exit_reason
            })
        
        trades_df = pd.DataFrame(trades_data)
        
        print(f"\n📋 Sample trades:")
        display(trades_df.head())
        
        # Basic statistics
        total_pnl = trades_df['pnl'].sum()
        winning_trades = len(trades_df[trades_df['pnl'] > 0])
        total_trades = len(trades_df)
        win_rate = winning_trades / total_trades * 100
        
        print(f"\n📈 Basic Statistics:")
        print(f"   Total Trades: {total_trades}")
        print(f"   Winning Trades: {winning_trades}")
        print(f"   Win Rate: {win_rate:.1f}%")
        print(f"   Total PnL: ₹{total_pnl:,.2f}")
        print(f"   Avg PnL per Trade: ₹{total_pnl/total_trades:,.2f}")
        
        # Plot equity curve
        trades_df_sorted = trades_df.sort_values('entry_time')
        trades_df_sorted['cumulative_pnl'] = trades_df_sorted['pnl'].cumsum()
        
        plt.figure(figsize=(12, 6))
        plt.plot(trades_df_sorted['entry_time'], trades_df_sorted['cumulative_pnl'])
        plt.title(f'{config.name} - Equity Curve for {test_symbol}')
        plt.xlabel('Date')
        plt.ylabel('Cumulative PnL')
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        # Exit reason analysis
        exit_reason_counts = trades_df['exit_reason'].value_counts()
        
        plt.figure(figsize=(8, 6))
        plt.bar(exit_reason_counts.index, exit_reason_counts.values)
        plt.title('Exit Reason Distribution')
        plt.xlabel('Exit Reason')
        plt.ylabel('Count')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
    else:
        print("⚠️ No trades generated")
else:
    print("❌ No data available for testing")

## Run Strategy on All Symbols

In [ ]:
# Run strategy on all symbols
if symbols:
    print(f"🚀 Running {config.name} on all symbols...")
    
    # Initialize strategy engine
    strategy_engine = StrategyEngine()
    
    # Run strategy
    success = strategy_engine.run_strategy(strategy, symbols)
    
    if success:
        print(f"✅ {config.name} completed successfully")
        
        # Load results
        results_df = strategy_engine.get_strategy_results(config.name)
        
        if results_df is not None and not results_df.empty:
            print(f"\n📊 Total Results:")
            print(f"   Total Trades: {len(results_df)}")
            print(f"   Symbols: {results_df['symbol'].nunique()}")
            print(f"   Date Range: {results_df['entry_time'].min()} to {results_df['entry_time'].max()}")
            
            # Display sample results
            display(results_df.head())
            
            # Basic statistics
            total_pnl = results_df['pnl'].sum()
            winning_trades = len(results_df[results_df['pnl'] > 0])
            total_trades = len(results_df)
            win_rate = winning_trades / total_trades * 100
            
            print(f"\n📈 Overall Statistics:")
            print(f"   Total Trades: {total_trades}")
            print(f"   Winning Trades: {winning_trades}")
            print(f"   Win Rate: {win_rate:.1f}%")
            print(f"   Total PnL: ₹{total_pnl:,.2f}")
            print(f"   Avg PnL per Trade: ₹{total_pnl/total_trades:,.2f}")
            
            # Symbol performance
            symbol_performance = results_df.groupby('symbol').agg({
                'pnl': ['count', 'sum', 'mean']
            }).round(2)
            
            print(f"\n📊 Top Performing Symbols:")
            symbol_performance.columns = ['Trades', 'Total PnL', 'Avg PnL']
            symbol_performance = symbol_performance.sort_values('Total PnL', ascending=False)
            display(symbol_performance.head(10))
            
        else:
            print(f"⚠️ No results found for {config.name}")
    else:
        print(f"❌ {config.name} failed")
else:
    print("❌ No symbols available")

## Analytics & Evaluation

In [ ]:
# Run analytics
if symbols:
    results_df = strategy_engine.get_strategy_results(config.name)
    
    if results_df is not None and not results_df.empty:
        print(f"📈 Running analytics for {config.name}...")
        
        # Initialize analytics engine
        analytics_engine = AnalyticsEngine()
        
        # Run comprehensive analysis
        result = analytics_engine.analyze_strategy(config.name, results_df)
        
        # Print detailed results
        print(f"\n📋 Detailed Analytics Results:")
        print(f"   Strategy: {result.strategy_name}")
        print(f"   Total Trades: {result.total_trades}")
        print(f"   Winning Trades: {result.winning_trades}")
        print(f"   Losing Trades: {result.losing_trades}")
        print(f"   Total PnL: ₹{result.total_pnl:,.2f}")
        print(f"   Max Drawdown: {result.max_drawdown*100:.2f}%")
        print(f"   Profit Factor: {result.profit_factor:.2f}")
        print(f"   Sharpe Ratio: {result.sharpe_ratio:.2f}")
        print(f"   Avg Duration: {result.avg_trade_duration:.1f} minutes")
        
        # Get recommendation
        recommendation = analytics_engine.evaluate_strategy(result)
        print(f"\n🎯 Recommendation: {recommendation}")
        
        if recommendation == "APPROVE":
            print("   ✅ Strategy approved for live trading")
        elif recommendation == "MODIFY":
            print("   ⚠️ Strategy needs modification")
        else:
            print("   ❌ Strategy killed - not viable")
        
        # Display metadata
        if result.metadata:
            print(f"\n📊 Additional Analytics:")
            
            # Win rate analysis
            if 'win_rate_analysis' in result.metadata:
                win_rate_data = result.metadata['win_rate_analysis']
                print(f"   Overall Win Rate: {win_rate_data['overall_win_rate']*100:.1f}%")
                print(f"   Symbol Avg Win Rate: {win_rate_data['symbol_avg_win_rate']*100:.1f}%")
            
            # Risk metrics
            if 'risk_metrics' in result.metadata:
                risk_data = result.metadata['risk_metrics']
                print(f"   VaR (95%): ₹{risk_data.get('var_95', 0):,.2f}")
                print(f"   Max Consecutive Losses: {risk_data.get('max_consecutive_losses', 0)}")
                print(f"   Avg Win: ₹{risk_data.get('avg_win', 0):,.2f}")
                print(f"   Avg Loss: ₹{risk_data.get('avg_loss', 0):,.2f}")
    else:
        print(f"❌ No results available for analytics")
else:
    print("❌ No data available")

## Summary

In [ ]:
print("\n" + "="*60)
print(f"🎯 {config.name} Strategy Analysis Complete")
print("="*60)

if symbols:
    results_df = strategy_engine.get_strategy_results(config.name)
    
    if results_df is not None and not results_df.empty:
        print(f"\n📊 Execution Summary:")
        print(f"   Symbols Processed: {results_df['symbol'].nunique()}")
        print(f"   Total Trades: {len(results_df)}")
        print(f"   Time Period: {results_df['entry_time'].min().date()} to {results_df['entry_time'].max().date()}")
        
        # Strategy characteristics
        print(f"\n🔧 Strategy Characteristics:")
        print(f"   Entry Logic: Opening Range Breakout")
        print(f"   Entry Window: {config.entry_time_start} - {config.entry_time_end}")
        print(f"   Exit Logic: Fixed time exit with stop loss/target")
        print(f"   Risk Management: {config.parameters['stop_loss_pct']*100}% stop loss")
        
        print(f"\n📁 Generated Files:")
        print(f"   📊 Results: itos/STR_results/{config.name}_all_trades.parquet")
        print(f"   📋 Analytics: itos/analytics/{config.name}_summary.csv")
        print(f"   📈 Dashboard: itos/analytics/{config.name}_dashboard.png")
    else:
        print(f"⚠️ No trades generated by {config.name}")
else:
    print("❌ Strategy execution failed")

print("\n" + "="*60)